# BERT tutorial using Hugging Face
## 教學目標
利用 Hugging Face 套件快速使用 BERT 模型來進行下游任務訓練
- 單一句型分類任務 (single-sentence text classification)

## 適用對象
已經有基本的機器學習知識，且擁有 Python、`numpy`、`pandas`、`scikit-learn` 以及 `PyTorch` 基礎的學生。

若沒有先學過 Python，請參考 [python-入門語法](https://github.com/IKMLab/course_material/blob/master/python-入門語法.ipynb) 教學。

若沒有先學過 `pandas`，請參考 [pandas-基本功能](https://github.com/IKMLab/course_material/blob/master/pandas-基本功能.ipynb) 教學。

若沒有先學過 `numpy`，請參考 [numpy-基本功能](https://github.com/IKMLab/course_material/blob/master/numpy-基本功能.ipynb) 教學。

若沒有先學過 `scikit-learn`，請參考 [scikit-learn-基本功能](https://github.com/IKMLab/course_material/blob/master/scikit-learn-基本功能.ipynb) 教學。

若沒有先學過  `PyTorch` ，請參考 [PyTorch-基本功能](https://github.com/IKMLab/course_material/blob/master/PyTorch-基本功能.ipynb) 教學。

若沒有先學過如何使用 `PyTorch` 建立自然語言處理序列模型，請參考 [NN-中文文本分類](https://github.com/IKMLab/course_material/blob/master/NN-中文文本分類.ipynb) 教學。

## BERT 簡易介紹
### Word embeddings 的問題
![Imgur](https://i.imgur.com/h6U5k41.png)
- 每個單詞的意思在不同的場合下應該有不同的意義表達
- 我們可以利用 RNN 作為語言模型，透過語言模型的輸入與輸出的處理來產生能夠理解上下文語意的 contextual embeddings
    - Language model: 語言模型，藉由估計(或最佳化)一整個序列的生成機率來輸出字詞的模型
        - 可以參考 [language model 的詳細教學](https://youtu.be/LheoxKjeop8?t=50)
- 藉由此種做法，我們可以將單詞語意的 word embeddings 轉換為具有上下文語意的 contextual embeddings

## 所以什麼是 BERT?
- 請參考理論層面的詳細教學 ([影片連結](https://www.youtube.com/watch?v=gh0hewYkjgo))
- 想進行 PyTorch 的 BERT 實作來獲得深入理解可以參考 ([網誌連結](https://leemeng.tw/attack_on_bert_transfer_learning_in_nlp.html))
- 也可以參考 Jay Alammar 的 The Illustrated BERT ([網誌連結](https://jalammar.github.io/illustrated-bert/))
- 也可以參考原始論文 ([論文連結](https://www.aclweb.org/anthology/N19-1423/))

### BERT 的 Pre-training 和 Fine-tuning 與先前方法比較
![Imgur](https://i.imgur.com/qfLhUaG.png)
- Pre-training 已經是 NLP 領域中不可或缺的方法
- 像 BERT 這類基於 Transformers 的[模型非常多](http://speech.ee.ntu.edu.tw/~tlkagk/courses/DLHLP20/BERT%20train%20(v8).pdf)，可以前往 [Hugging Face models](https://huggingface.co/models) 一覽究竟

## Hugging Face 介紹
- 🤗 Hugging Face 是專門提供自然語言處理領域的函式庫
- 其函式庫支援 PyTorch 和 TensorFlow
- 🤗 Hugging Face 的主要套件為:
    1. Transformers ([連結](https://huggingface.co/transformers/index.html))
    - 提供了現今最強大的自然語言處理模型，使用上非常彈性且方便
    2. Tokenizers ([連結](https://huggingface.co/docs/tokenizers/python/latest/))
    - 讓你可以快速做好 BERT 系列模型 tokenization
    3. Datasets ([連結](https://huggingface.co/docs/datasets/))
    - 提供多種自然語言處理任務的資料集

In [ ]:
# !pip install torch==2.4.0
# !pip install transformers==4.37.0
!pip install datasets
# !pip install accelerate==0.21.0
# !pip install scikit-learn==1.5.2
!pip install wget
# !pip install tarfile

In [ ]:
# 1. Check the versions of your packages
import torch
print(f"PyTorch version: {torch.__version__}")

import transformers
print(f"Hugging Face Transformers version: {transformers.__version__}")

import datasets
print(f"Hugging Face Datasets version: {datasets.__version__}")

In [ ]:
import os
import json
import numpy as np
from pathlib import Path # (Python3.4+)

# 單一句型分類任務 (single-sentence text classification)
## 準備資料集 (需先下載)
我們使用 IMDb reviews 資料集作為範例

In [ ]:
# 下載 IMDb 資料集
import wget
url = 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
filename = wget.download(url, out='./')

In [ ]:
# 解壓縮 IMDb 資料集

import tarfile

# 指定檔案位置，並解壓縮 .gz 結尾的壓縮檔
tar = tarfile.open('aclImdb_v1.tar.gz', 'r:gz')
tar.extractall()

## 接下來我們要進行資料前處理
但首先要觀察解壓縮後的資料夾結構:
```
aclImdb---
        |--train
        |    |--neg
        |    |--pos
        |    |--...
        |--test
        |    |--neg
        |    |--pos
        |    |--...
        |--imdb.vocab
        |--imdbEr.text
        |--README
```
其中 train 和 test 資料夾中分別又有 neg 和 pos 兩種資料夾

我們要針對這兩個目標資料夾進行處理

In [ ]:
# Create a function to pre-process the IMDb dataset
def read_imdb_split(split_dir):
    split_dir = Path(split_dir)
    texts, labels = [], []
    for label_dir in ["pos", "neg"]:
        # Use glob() to get files with the extension ".txt"
        for text_file in (split_dir/label_dir).glob("*.txt"):
            # read_text() returns the decoded contents of the pointed-to file as a string
            tmp_text = text_file.read_text()

            # Append the read text to the list we defined in advance
            texts.append(tmp_text)

            # Build labels based on the folder name
            labels.append(0 if label_dir == "neg" else 1)

    return texts, labels

In [ ]:
# Pre-process the IMDb dataset (execution)

train_texts, train_labels = read_imdb_split('aclImdb/train')
test_texts, test_labels = read_imdb_split('aclImdb/test')

### 切分訓練資料，來分出 validation set

In [ ]:
# Use train_test_split to split the training data into training and validation data
from sklearn.model_selection import train_test_split

# Set a random seed for reproducibility
random_seed = 42

# Set the ratio of the validation set to the training set
valid_ratio = 0.2

train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts,
    train_labels,
    test_size=valid_ratio,
    random_state=random_seed
)

## 輸入 BERT 的前處理
![Imgur](https://i.imgur.com/3C7xDlf.png)
(圖片來源: BERT [原始論文](https://www.aclweb.org/anthology/N19-1423/))

### Tokenization
- 斷字的部份以 DistilBERT (Sanh et al., 2019) 的 tokenizer 為例
- Hugging Face 的 tokenizer 可以直接幫你自動將資料轉換成 BERT 的輸入型式 (也就是加入[CLS]和[SEP] tokens)

## Hugging Face AutoTokenizer
- 使用 AutoTokenizer 搭配 Hugging Face models 的名稱可以直接呼叫使用
- 舉例:
    - transformers.AutoTokenizer.from_pretrained('distilbert-base-uncased')
    - 等同於 transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
- [點這裡來查看 Hugging Face models 的名稱](https://huggingface.co/transformers/pretrained_models.html)

In [ ]:
# Load the Hugging Face tokenizer

model_name = "bert-base-uncased"
# Use .from_pretrained() for a pre-trained model
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

In [ ]:
# Perform tokenization for the train / val / test datas
# truncation: 代表依照 max_length 進行序列長度的裁切
# max_length 可以在 tokenizer 的 parameters 中進行設定
# 如果沒有指定 max_length，則依照所使用的模型的序列最大長度
# padding 為 True 表示會將序列長度補齊至該 batch 的最大長度 (欲知詳情請查看 source code)

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

# 但這樣的話所有的句子都會被 padded 到一樣的長度

In [ ]:
# 查看 max_length

tokenizer.model_max_length

In [ ]:
# 查看 tokenizer 的功能

tokenizer_methods = dir(tokenizer)

# Group them into rows for better readability
for i in range(0, len(tokenizer_methods), 5):
    print(", ".join(tokenizer_methods[i:i+5]))
    tokenizer_methods = dir(tokenizer)

In [ ]:
# 查看 [CLS] token 和 [SEP] token 在字典中的 ID

print("The ID of [CLS] token is {}.".format(tokenizer.vocab["[CLS]"]))
print("The ID of [SEP] token is {}.".format(tokenizer.vocab["[SEP]"]))
print("The ID of [PAD] token is {}.".format(tokenizer.vocab["[PAD]"]))

In [ ]:
# 查看 output encodings 的 key 值內容

print(val_encodings.keys())

In [ ]:
# 查看 output encodings 是什麼型態

print(type(val_encodings))

### 檢查 tokenization 後的結果
- 使用 Hugging Face tokenizer 進行 tokenization 後的結果是一個 dict
- 這個 dict 的 keys 包含 'input_ids' 和 'attention_mask'
- input_ids: 原本句子中的每個字詞被斷詞後轉換成字典的 ID
    - 注意!! tokenizer 小小的動作已經幫你完成了斷詞和 word to ID 的轉換
- attention_mask: tokenization 後句子中包含文字的部分為 1，padding 的部分為 0
    - 可以想像成模型需要把注意力放在有文字的位置

In [ ]:
# 檢查 tokenization 後的結果

print(val_encodings.input_ids[0])
print(val_encodings.token_type_ids[0])
print(val_encodings.attention_mask[0])

In [ ]:
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        # Note that the tokenizer output is a dict wrapper
        # Convert data and labels into PyTorch tensors
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])

        return item

    def __len__(self):
        # Number of a dataset
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

### 除了自己處理資料，你還可以使用 Hugging Face Datasets
- Hugging Face Datasets 已經幫你收錄了自然語言處理領域常見的資料集
- 直接呼叫 Datasets 並搭配下面幾個 cells 的語法，可省下不少時間
- 但前提是你要進行的任務資料集有被收錄在 Hugging Face Datasets

In [ ]:
# Load the IMDb training set
train = datasets.load_dataset("imdb", split="train")

# Split the validation set
random_seed = 42
splits = train.train_test_split(
    test_size=0.2,
    seed=random_seed
)
train, valid = splits['train'], splits['test']

# Load the IMDb test set
test = datasets.load_dataset("imdb", split="test")

In [ ]:
def to_torch_data(hug_dataset):
    """Transform Hugging Face Datasets into PyTorch Dataset
    Args:
        - hug_dataset: data loaded from HF Datasets
    Return:
        - dataset: PyTorch Dataset
    """
    dataset = hug_dataset.map(
        lambda batch: tokenizer(
            batch["text"],
            truncation=True,
            padding=True
        ),
        batched=True
    )
    # dataset.set_format(
    #     type='torch',
    #     columns=[
    #         'input_ids',
    #         'attention_mask',
    #         'label'
    #     ]
    # )
    return dataset

In [ ]:
train_dataset = to_torch_data(train)
val_dataset = to_torch_data(valid)
test_dataset = to_torch_data(test)

In [ ]:
print(train_dataset.format['type'])

## 使用 Hugging Face 的模型
- 在這個 API 盛行的世代，總是有人幫你設想周到
- [Hugging Face 的模型頁面連結](https://huggingface.co/models)
- 以 BERT 為例，只要透過 AutoModel.from_pretrained("bert-base-uncased")，就可以直接使用 BertModel
- 需要注意的是接下來你要做怎樣的下游任務訓練
- 同樣以 BERT 為例，在原始論文中 BERT 進行過以下的任務:
    - Sentence pair classification: MNLI/QQP/QNLI/MRPC/RTE/WNLI
        - 對應 `BertForSequenceClassification`
        - 使用雙句結合，並以分類的方式進行訓練
    - Semantic textual similarity: STS-B
        - `BertForSequenceClassification`
        - 使用雙句結合，並以迴歸的方式進行訓練
    - Single sentence classification: SST-2/CoLA
        - 對應 `BertForSequenceClassification`
        - 使用單句，並以迴歸的方式進行訓練
    - Question answering: SQuAD v1.1/v2.0
        - 對應 `BertForQuestionAnswering`
        - 使用雙句(問題+原文)，並透過答案在原文中的位置進行訓練
    - Named-entity recognition (slot filling): CoNLL-2003
        - 對應 `BertForTokenClassification`
        - 使用單句，並以分類的方式進行訓練
- 如果要進行的下游任務訓練不在 Hugging Face 已經建好的模型範圍，那就需要自己寫一個 model class:
    1. 繼承 torch.nn.Module
    2. 利用 super 來繼承所有親屬類別的實體屬性
    3. 定義欲使用的 pre-trained model
    4. 定義會使用到的層如 linear 或 Dropout 等
    5. 設計 forward function 並且設定下游任務的輸出

In [ ]:
# 利用 AutoModel 呼叫模型
model = transformers.AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=3,
    # problem_type="single_label_classification"
)

In [ ]:
print(model.config.problem_type)

## 進行模型的訓練
### 使用 Hugging Face Trainer ([Documentation](https://huggingface.co/transformers/main_classes/trainer.html))
- Trainer 是 Hugging Face 中高度封裝的套件之一，負責模型訓練時期的"流程"
- 過去我們自行寫訓練流程的程式碼可以交給 Trainer
- Trainer 需要搭配使用 [TrainingArguments](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments)
    - TrainingArguments 是 Trainer 所需要的引數

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
!pip install --upgrade accelerate

In [ ]:
training_args = transformers.TrainingArguments(
    output_dir='./results',            # 輸出的資料夾
    num_train_epochs=3,                # 總共要訓練多少 epochs
    learning_rate=2e-5,                # learning rate
    per_device_train_batch_size=16,    # training 時的 batch size，if 2 GPUs: 32
    per_device_eval_batch_size=64,     # test 時的 batch size，if 2 GPUs: 128
    gradient_accumulation_steps=2,     # 梯度累積次數 (代表多少個 steps 才會更新一次模型)
    lr_scheduler_type='linear',        # Learning rate 在 warmup_steps 中上升後下降
    warmup_steps=500,                  # Learning rate 從零緩慢上升的 steps
    weight_decay=0.01,                 # hyperparameter for optimizer
    evaluation_strategy='steps',       # time unit to perform evaluation
    save_strategy='steps',             # time unit to save checkpoints
    save_steps=500,                    # how often to save checkpoints
    eval_steps=500,                    # how often to perform evaluation
    load_best_model_at_end=True,       # if loading the best checkpoint at the end of training
    metric_for_best_model='eval_loss', # how to judge the best model
    report_to='tensorboard',           # if saving TensorBoard records
    save_total_limit=10,               # maximum number of saved checkpoints
    logging_dir='./logs',              # folder for logs
    logging_steps=10,                  # how often to save logs
    seed=random_seed                   # for reproducibility control
)

trainer = transformers.Trainer(
    model=model,                         # 🤗 model
    args=training_args,                  # the `TrainingArguments` you set
    train_dataset=train_dataset,         # the training dataset
    eval_dataset=val_dataset,            # the evaluation dataset
    compute_metrics=compute_metrics      # evaluation metric
)

# Use 1 GPU for training
trainer.args._n_gpu=1

# start training
trainer.train()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./logs

In [ ]:
# 測試模型

trainer.predict(test_dataset)